In [1]:
import os
import sys
import mysql.connector

from pathlib import Path
from openai import OpenAI

current_dir = Path().resolve()
# If we're in the basics directory, go up one level; otherwise use current directory
if current_dir.name == 'basics':
    parent_dir = current_dir.parent
else:
    parent_dir = current_dir

if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from _utils.standard_functions import load_env

In [2]:
# reading the env
load_env()

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv("OPENAI_BASE_URL")

OpenAI API Key exists and begins with sk-proj-
Google API Key exists and begins with AI
Anthropic API Key not set
DeepSeek API Key not set
Groq API Key not set
Grok API Key not set
OpenRouter API Key not set


In [3]:
# set DB connection variables
DB_HOSTNAME = os.getenv('DB_HOSTNAME')
DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_SCHEMA = os.getenv('DB_SCHEMA')
DB_PORT = os.getenv('DB_PORT')

In [10]:
def execute_sql(sql_query):
    conn = mysql.connector.connect(
        host=DB_HOSTNAME,
        user=DB_USERNAME,
        password=DB_PASSWORD,
        database=DB_SCHEMA,
        port=DB_PORT
    )
    cursor = conn.cursor()
    cursor.execute(sql_query)
    
    # Check if query is a SELECT (returns data) or DML (needs commit)
    query_upper = sql_query.strip().upper()
    if query_upper.startswith('SELECT'):
        # For SELECT queries, fetch and return results
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description] if cursor.description else []
        cursor.close()
        conn.close()
        return {'columns': columns, 'rows': results}
    else:
        # For INSERT/UPDATE/DELETE, commit the transaction
        affected_rows = cursor.rowcount
        conn.commit()
        cursor.close()
        conn.close()
        return {'affected_rows': affected_rows}

In [11]:
# test execution
sql_query="SELECT id, random_column, random_foreign_key FROM llm_core.test"
execute_sql(sql_query)

{'columns': ['id', 'random_column', 'random_foreign_key'],
 'rows': [(1, 'sdgfsd', 'sdfgfds'),
  (2, 'sdf', 'sdf'),
  (3, 'fd', 'hgf'),
  (4, 'jhgf', 'ert')]}